In [1]:
from scipy.optimize import milp, LinearConstraint, Bounds
import numpy as np
from utils import sedaroLogin, contact_booleans_to_intervals, selected_contacts_to_schedule
from typing import Any
import time

# Settings
scenario_branch = 'PPMjttdp2FbPcvfxSScHgB'
ground_segment_template = 'PPHLMTljknVcsY6pnhJ5Qf' # Read-only
gs_agent_name = 'Atlas (Enterprise)'
minimum_uplink = 3*240000000 # bits

# Constants
RESOLUTION_SECONDS = 10. # Sedaro provides a handle for scheduling contacts at 10s resolution

client = sedaroLogin()
scenario = client.scenario(scenario_branch)
template = client.agent_template(ground_segment_template)
uplink_bitrate = template.ScheduledTransmitInterface.get_first().onBitRate


## Set up scheduling as an Integer Linear Programming Problem

In [2]:
def optimize_schedule(
        # projected_contacts: dict[tuple[str, str], list[bool]],
        # tg_targets: list[tuple[str, list[str]]]
        contact_intervals: list[tuple[int, int]],
        c_location: dict[str, list[int]],
        c_tg: dict[str, list[int]],
        durations: list[int],
) -> tuple[dict[str, Any], ...]:
    '''
    Based on reference [1], set up the contact scheduling as an integer linear programming problem, then solve using
    scipy.optimize.milp.

    Args:
        contact_intervals: A list of all contacts, represented as a tuple of start and stop indices
        c_location: A dictionary of contact indices for each location
        c_tg: A dictionary of contact indices for each target group
        durations: The duration of each contact

    [1] Eddy, D., Ho, M., and Kochenderfer, M. J., “Optimal Ground Station Selection for Low-Earth Orbiting Satellites”, 
        arXiv e-prints, Art. no. arXiv:2410.16282, 2024. doi:10.48550/arXiv.2410.16282.
    '''
    n_contacts = len(contact_intervals)
    # The solution vector x is a list of binaries of length 2*len(C). The first len(C) selects contacts for downlink
    # and the second len(C) selects contacts for uplink. We will maximize data downlink while satisfying the minimum 
    # uplink requirements. 

    # First, well set up the objective function. The amount of data linked down is proportional to link duration
    f = -np.hstack([np.array(durations), np.zeros(n_contacts)])

    # Next, we set up the constraints.
    constraints = []
    # The first constraint is the aforementioned minimum uplink requirement
    A = np.hstack([np.zeros(n_contacts), np.array(durations)])
    minimum_uplink_steps = minimum_uplink / uplink_bitrate / RESOLUTION_SECONDS
    constraints.append(LinearConstraint(A, lb=minimum_uplink_steps))

    # The second is that each antenna can only communicate with one spacecraft at a time, based on eqn. 11
    # The third is like it. Eddy et al. require each spacecraft only communicate with one antenna at a time (eqn. 12),
    # but we require the stronger condition that each target group communicate with one antenna at a time. These are 
    # similar constraints, so we set up a helper function
    def exclusion_matrix(entity_contacts):
        '''
        Returns a matrix for evaluating exclusion constraints. Each row will find the sum of conflicting contacts. This
        matrix can be used to create an efficient LinearConstraint to enforce A@x <= 1.
        '''
        exclusion_rows = []
        for contact_indices in entity_contacts.values():
            flattened_schedule_indices = sorted([(contact_intervals[i], i) for i in contact_indices], key=lambda x: x[0][0])
            flat_schedule, sorted_indices = zip(*flattened_schedule_indices)
            for i in range(1, len(flat_schedule)):
                if flat_schedule[i][0] < flat_schedule[i-1][1]:
                    # These contacts overlap, so we must make a constraint that only one of them is selected, including uplink
                    contact1_index = sorted_indices[i]
                    contact2_index = sorted_indices[i-1]
                    A = np.zeros(2*n_contacts)
                    A[contact1_index] = 1
                    A[contact2_index] = 1
                    A[contact1_index+n_contacts] = 1
                    A[contact2_index+n_contacts] = 1
                    exclusion_rows.append(A)
        return np.vstack(exclusion_rows)
    
    # Add constraint if any location overlaps exist
    if station_exclusion_rows := exclusion_matrix(c_location):
        constraints.append(LinearConstraint(station_exclusion_rows, ub=1))
    # Same for target groups
    if tg_exclusion_rows := exclusion_matrix(c_tg):
        constraints.append(LinearConstraint(tg_exclusion_rows, ub=1))

    # Run the solver and time it for fun
    start = time.time()
    result = milp(
        f, # f@x is minimized
        integrality=np.ones(2*n_contacts), # all params are integers
        bounds=[Bounds(0, 1) for _ in range(2*n_contacts)], # all params are binary
        constraints=constraints
        ) 
    end = time.time()
    print(f"Optimization took {end-start} seconds. {result.message}")

    return result.x
    
    

## Set up cosimulated external state

In [3]:
# clear out existing blocks
scenario.delete_all_external_state_blocks()

# Get the ground segment agent
gs_agent = scenario.Agent.get_where(name=gs_agent_name)[0]

# The existing scheduler will provide us a handle for the the contacts, and we can overwrite the schedule
scheduler = template.ContactScheduler.get_first()
external_state = scenario.PerRoundExternalState.create(
    consumed = [
        {f'block.{scheduler.id}': ['projectedContacts', 'targetSeries']},
        {f'block.{scheduler.id}.interfaces': ['id', 'type']},
        {f'block.{scheduler.id}.interfaces.linkTargetGroup': ['id', 'targets.id', 'targets.agentId']},
        'time'
        ],
    produced = [{f'block.{scheduler.id}': 'schedule'}],
    engineIndex = 0,
    agents = [gs_agent.id]
)
ancillary_state = scenario.PerRoundExternalState.create(
    consumed = [{'elapsedTime.as': 'Duration.hour'}],
    produced = [{f'block.{scheduler.id}': '_generateNewSchedule'}],
    engineIndex = 0,
    agents = [gs_agent.id]
)

## Cosimulate

In [ ]:
schedule_period = 4 # hours
schedules_published = 0
schedule = None

import pickle
def backup(var):
    backup = 'savestate.pkl'
    with open(backup, 'wb') as f:
        pickle.dump(var, f)

with scenario.simulation.start(wait=True) as simulation_handle:
    async with simulation_handle.async_channel() as channel:
        print('Simulation started!')
        ancillary_data = await channel.consume(agent_id=gs_agent.id, external_state_id=ancillary_state.id)
        t_h = ancillary_data[0]
        print(t_h)
        if t_h // schedule_period > schedules_published:
            print("Generating new schedule!")
            # Generate the schedule
            schedules_published += 1
            # Set _generateNewSchedule to True to trigger contact projection
            await channel.produce(agent_id=gs_agent.id, external_state_id=ancillary_state.id,values=(True,))
            # Get the necessary inputs
            print("Getting inputs...")
            schedule_inputs = await channel.consume(agent_id=gs_agent.id, external_state_id=external_state.id)
            (projected_contacts, target_series), interface_ids, tg_targets, mjd = schedule_inputs
            backup([projected_contacts, tg_targets])
            # Format for optimizer
            print('Setting up optimizer...')
            c_t, contact_intervals, c_location, c_satellite, c_tg, durations, contact_labels = contact_booleans_to_intervals(projected_contacts, tg_targets)
            # This runs the optimizer which selects contacts, which we will parse into a schedule
            backup([contact_intervals, c_location, c_tg, durations])
            contacts_selected = optimize_schedule(contact_intervals, c_location, c_tg, durations)
            backup([contacts_selected, contact_intervals, contact_labels, target_series, interface_ids, tg_targets, mjd])
            # Parse the optimizer output into a schedule
            print('Parsing optimizer output...')
            schedule = selected_contacts_to_schedule(
                contacts_selected, 
                contact_intervals,
                contact_labels,
                target_series,
                interface_ids,
                tg_targets,
                mjd,
                RESOLUTION_SECONDS,
                )
            await channel.produce(agent_id=gs_agent.id, external_state_id=ancillary_state.id, values=(schedule,))
            print('Done!')
        else:
            await channel.produce(agent_id=gs_agent.id, external_state_id=ancillary_state.id, values=(False,))
            await channel.produce(agent_id=gs_agent.id, external_state_id=external_state.id, values=(schedule,))